In [3]:
from keras.layers import *
from keras.models import Sequential, load_model
import numpy as np
np.random.seed(7)
from scanner import *
from localizer import *
from tools import *
from models import *
from keras.callbacks import *
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Using TensorFlow backend.


In [8]:
wifi = wifi_scanner('wlp3s0')

lab_loc = localizer(wifi)
lab_loc.load_profile('lab_profile')

X_train, y_train, X_test, y_test = load_data_from_folder('../datasets', lab_loc.profile, train_test_split=0.9, randomize=True, item='rssi')
print(X_train.shape)
keys = []
for item in y_train:
    if str(item) not in keys:
        keys.append(str(item))
print(keys)
loc_dict = {}
index_dict = {}
for i, item in enumerate(keys):
    loc_dict[item] = i
    index_dict[i] = item
print(index_dict)
print(loc_dict['[ 0.5  0.5]'])

Loaded profile: lab_profile


2017-06-28_13:52 loaded
Loaded 1 files from ../datasets
(810, 20)
['[ 1.  0.]', '[ 1.   0.5]', '[ 0.5  0.5]', '[ 1.  1.]', '[ 0.  0.]', '[ 0.   0.5]', '[ 0.5  0. ]', '[ 0.  1.]', '[ 0.5  1. ]']
{0: '[ 1.  0.]', 1: '[ 1.   0.5]', 2: '[ 0.5  0.5]', 3: '[ 1.  1.]', 4: '[ 0.  0.]', 5: '[ 0.   0.5]', 6: '[ 0.5  0. ]', 7: '[ 0.  1.]', 8: '[ 0.5  1. ]'}
2


In [17]:
# assign each of the nine locations a class
y_train_clf = np.zeros((X_train.shape[0], len(loc_dict)))
for i, item in enumerate(y_train):
    y_train_clf[i, loc_dict[str(item)]] = 1

y_test_clf = np.zeros((X_test.shape[0], len(loc_dict)))
for i, item in enumerate(y_test):
    y_test_clf[i, loc_dict[str(item)]] = 1

(810, 9)

In [119]:
model = Sequential()
model.add(Dense(1024, kernel_initializer='he_normal', input_shape=(20,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(256, kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(9, kernel_initializer='he_normal'))
model.add(Activation('sigmoid'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train_clf, epochs=4, batch_size=32, verbose=0)
print(model.evaluate(X_train, y_train_clf, batch_size=32, verbose=0))
print(model.evaluate(X_test, y_test_clf, batch_size=32, verbose=0))

[0.94969030765839568, 0.6456790121985071]
[0.97676166163550482, 0.63333333068423803]


In [121]:
x2, y2 = load_data_from_file('../datasets/old_datasets/2017-07-06_08:46', lab_loc.profile, item='rssi')
print(x2.shape)
# all values are x=0, y=1
preds = model.predict(x2)
for pred in preds:
    print(index_dict[np.argmax(pred)])

(100, 20)
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5  0. ]
[ 0.5 

In [63]:
for i in range(10):
    cells = np.expand_dims(lab_loc.wifi.get_wifi_cells(lab_loc.profile, item='rssi'), axis=0)
    cells = scale_inputs(cells)
    preds = model.predict(cells)
    print(preds)
    print(np.argmax(preds))
    print(index_dict[np.argmax(preds)])

[[  2.46785174e-04   6.49177507e-02   1.80426165e-02   3.28637514e-04
    1.04520259e-04   4.77886942e-07   1.33276204e-04   1.16700699e-06
    1.31487264e-04]]
1
[ 1.   0.5]


[[  9.94084752e-04   2.66363490e-02   3.57921869e-02   9.28085065e-04
    4.07329790e-04   7.22132609e-06   2.15702879e-04   1.26373288e-05
    2.48284242e-03]]
2
[ 0.5  0.5]


[[  5.45362569e-03   2.47784019e-01   1.86703093e-02   2.43726466e-03
    7.43085111e-05   1.58561750e-06   6.92009780e-05   5.48376847e-05
    2.43485774e-04]]
1
[ 1.   0.5]


[[  3.04237474e-03   1.29848331e-01   1.81942843e-02   1.86440162e-03
    3.64603147e-05   1.00029638e-06   5.28046330e-05   1.18183407e-05
    1.82504024e-04]]
1
[ 1.   0.5]


[[  1.38959475e-02   2.78452318e-02   1.21920578e-01   1.25688978e-03
    7.48196107e-05   3.65755150e-05   1.95990637e-04   1.24968879e-04
    2.51002912e-03]]
2
[ 0.5  0.5]


[[  2.41677966e-02   2.79321875e-02   1.17738552e-01   1.48381921e-03
    2.21988375e-05   2.69456232e-05   1.16882780e-04   9.22269101e-05
    1.06202753e-03]]
2
[ 0.5  0.5]


[[  2.08939780e-02   3.82719487e-02   9.03329924e-02   1.91901287e-03
    4.20677898e-05   2.19897611e-05   1.05479245e-04   1.22341531e-04
    1.54899259e-03]]
2
[ 0.5  0.5]


[[  1.33606410e-02   4.63113897e-02   4.56377640e-02   2.39202101e-03
    3.20458930e-05   7.61379215e-06   5.78526706e-05   5.69260919e-05
    8.57896986e-04]]
1
[ 1.   0.5]


[[  1.41846901e-02   6.72299564e-02   4.71563824e-02   2.16607843e-03
    1.87932274e-05   4.98488134e-06   5.59870750e-05   4.17605333e-05
    3.79598932e-04]]
1
[ 1.   0.5]


[[  6.70276722e-03   8.60758200e-02   3.39152142e-02   1.96090923e-03
    1.92448879e-05   2.18778564e-06   5.35500767e-05   1.69840459e-05
    2.32685488e-04]]
1
[ 1.   0.5]
